In [47]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.onnx
import torch
import onnx
import numpy as np
import onnxruntime as ort
from workload_generator import parse_onnx, write_csv



In [48]:

# Load the pretrained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Attach hook
def store_input_hook(module, input, output):
    module.input_hooked = input

model = GPT2LMHeadModel.from_pretrained(model_name)
hooked = {}
# Do a recursive search for all modules that are of type "lookup_modules" and register the hook
def register_hook(module, hook, lookup_modules = [torch.nn.Linear]):
    for child in module.children():
        if isinstance(child, tuple(lookup_modules)):
            handle = child.register_forward_hook(store_input_hook)
            hooked[child] = handle
        else:
            register_hook(child, store_input_hook, lookup_modules)

register_hook(model, hooked, lookup_modules = [torch.nn.Linear, type(model.transformer.h[0].mlp.c_fc)])

print(hooked)


# Example usage
input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)


# Set the path for saving the ONNX file
onnx_file_path = "model.onnx"

# Set the input tensor
input_ids = torch.tensor([[15496, 11, 703, 389, 345, 30]])

# Export the model to ONNX
torch.onnx.export(model, input_ids, onnx_file_path)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f3070f0f1f0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524e50>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f5244f0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f5244c0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524460>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f5243d0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524370>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524310>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524190>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524220>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f5241c0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524070>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f524100>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306f

/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, how are you?

I'm a little bit of a nerd. I'm a


In [30]:
keys = [k.input_hooked[0] for k in hooked.keys()]

In [53]:
in_A = keys[0][0,0]
in_B = list(hooked.keys())[0].weight[:,:4]

In [64]:
# Quantize in_B with max_in_B = 7,
# min_in_B = -8
max_in_B = in_B.max()
min_in_B = in_B.min()
scale_in_B = (max_in_B - min_in_B) / 15
in_B = torch.round(in_B / scale_in_B)


In [70]:
in_B.min()

tensor(-8.)

In [65]:
ref_out = in_A @ in_B

In [66]:
ref_out

tensor([-0.6352,  3.1979,  8.6187, -4.9695], grad_fn=<SqueezeBackward4>)

In [67]:
# Save to torch file
torch.save({
    'in_A': in_A,
    'in_B': in_B,
    'ref_out': ref_out
}, 'test_data.pth')

In [ ]:
# Load the ONNX model
onnx_model = onnx.load("model.onnx")


In [ ]:
# Parse the ONNX model
parsed_model, other_ops = parse_onnx(onnx_model)

In [ ]:
pdict = {k: v for k,v in parsed_model.items() if v.op_type == 'Gemm' or v.op_type == 'MatMul'}
pdict['transformer.h.1.attn.MatMul']

In [ ]:
# Write the parsed model to a CSV file
write_csv(parsed_model, "gpt2_gen.csv")

In [ ]:
import torch 
